In [1]:
# Client ID: 24efe6738ba9459fad737a032f1f8a06
# Client secret: d15917479fb24211926472b6c6a67882

## 0. Import libraries

In [2]:
import requests
import base64

import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

## 1.0 API Connfiguration

In [3]:
# Replace with your own Client ID and CLient Secret

CLIENT_ID = '24efe6738ba9459fad737a032f1f8a06'
CLIENT_SECRET = 'd15917479fb24211926472b6c6a67882'

In [4]:
# Base64 encode the client ID and client secret

client_credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [5]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'

In [6]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


Definición del cliente ID y cliente secret:
Se inicializan las variables CLIENT_ID y CLIENT_SECRET con valores ficticios ('your_client_id' y 'your_client_secret'). En tu caso, deberías reemplazarlos con tus propios valores proporcionados por Spotify al registrar tu aplicación en su plataforma de desarrolladores.

Codificación en Base64 del cliente ID y cliente secret:
La variable client_credentials combina el cliente ID y el cliente secret en un string separado por :. Luego, se codifica en Base64 usando base64.b64encode() para generar las credenciales codificadas. Estas credenciales se utilizan más adelante en la solicitud para la autorización.

Solicitud del token de acceso:
Se establece la URL del endpoint de autenticación de Spotify como token_url. Se configuran los encabezados para la solicitud POST, incluyendo la autenticación básica con las credenciales codificadas en Base64. El cuerpo de la solicitud contiene el tipo de concesión ('client_credentials').

Envío de la solicitud y gestión de la respuesta:
Se realiza la solicitud POST a la URL de token con los datos y encabezados configurados. Si la solicitud es exitosa (código de estado 200), se extrae el token de acceso de la respuesta JSON y se imprime un mensaje indicando que se ha obtenido el token correctamente. De lo contrario, si hay un error en la solicitud, se muestra un mensaje de error y el programa finaliza.

## 2.0 Function definition to get playlist and dataset creation

In [7]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)
        
    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

The function begins by initializing the Spotipy client with the provided access_token, which serves as the authentication token to interact with the Spotify Web API. The access_token allows the function to make authorized requests to access Spotify’s resources. The function then uses the Spotipy client to fetch information about the tracks in the specified playlist (identified by its playlist_id). The sp.playlist_tracks method retrieves the playlist tracks. The fields parameter is used to specify the specific track information that is required, such as track ID, name, artists, album ID, and album name.

The function then extracts relevant information from the retrieved playlist tracks and stores it in a list of dictionaries called music_data. For each track in the playlist, the function extracts data such as track name, artists (combined into a single string), album name, album ID, track ID, and popularity. The function uses the sp.audio_features method to fetch audio features for each track in the playlist. These audio features include attributes like danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, etc. These audio features provide insights into the characteristics of each track.

The extracted information for all tracks is stored in the music_data list. The function then creates a DataFrame from the music_data list. The DataFrame organizes the music data in a tabular format, making it easier to analyze and work with the collected information.

In [8]:
playlist_id1 = '37i9dQZF1DX76Wlfdnj7AP'

In [9]:
# Call the function to get the music data from the playlist and store it in a DataFrame
music_df1 = get_trending_playlist_data(playlist_id1, access_token)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=59NraMJsLaMCVtwXTSia8i:
 Max Retries, reason: too many 429 error responses

In [ ]:
# Display the DataFrame

print(music_df1)

In [ ]:
playlist_id2 = '5lUvOTERh8HpzDLv1L1nfs'
music_df2 = get_trending_playlist_data(playlist_id2, access_token)

In [ ]:
music_df2.shape

## 2.1 Second version of the function

In [ ]:
# This second version of the function takes all the songs of the list instead of only 100.

In [ ]:
def get_trending_playlist_data(playlist_id, access_token):
    sp = spotipy.Spotify(auth=access_token)
    offset = 0
    limit = 100
    music_data = []

    while True:
        playlist_tracks = sp.playlist_tracks(
            playlist_id, fields='items(track(id, name, artists, album(id, name)))', offset=offset, limit=limit)

        if not playlist_tracks['items']:
            break

        for track_info in playlist_tracks['items']:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

            try:
                album_info = sp.album(album_id) if album_id != 'Not available' else None
                release_date = album_info['release_date'] if album_info else None
            except:
                release_date = None

            try:
                track_info = sp.track(track_id) if track_id != 'Not available' else None
                popularity = track_info['popularity'] if track_info else None
            except:
                popularity = None

            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': track_info.get('explicit', None),
                'External URLs': track_info.get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
            }

            music_data.append(track_data)
        
        offset += limit

    df = pd.DataFrame(music_data)

    return df


In [ ]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'
music_df = get_trending_playlist_data(playlist_id, access_token)

In [ ]:
music_df.shape

In [ ]:
playlist_id = '5lUvOTERh8HpzDLv1L1nfs'
music_df_2 = get_trending_playlist_data(playlist_id, access_token)

In [ ]:
music_df_2.shape

In [ ]:
# I want use 2 combined playlist to get the recommendation system. I get separatly the 2 dataframes and the I merged them in one.
final_df = pd.concat([music_df, music_df_2])

In [ ]:
final_df.shape

In [ ]:
final_df.info()

In [10]:
music_df = final_df
data = music_df

NameError: name 'final_df' is not defined

## 3.0 Building the music recommendation system

### 3.1 Function to calculate weighted popularity
While providing music recommendations to users, it is important to recommend the latest releases. For this, we need to give more weight to the latest releases in the recommendations. Let’s write a function to solve this problem:

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

### 3.2 Normalizing music features

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
music_df.head()

In [ ]:
music_features_scaled

### 3.1 Function to get content-based recommendations based on music features


In [ ]:
# A function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

The above function takes input_song_name as the input, which represents the name of the song for which recommendations are to be generated. The function checks if the input_song_name exists in the music_df DataFrame, which presumably contains the music data with features like ‘Track Name’, ‘Artists’, ‘Album Name’, ‘Release Date’, and ‘Popularity’. If the input song name is found in the music_df DataFrame, the function retrieves the index of the input song in the DataFrame. This index will be used to compare the audio features of the input song with other songs in the dataset.

The function calculates the similarity scores between the audio features of the input song and all other songs in the dataset. It uses cosine similarity, a common measure used in content-based filtering. The cosine_similarity function from scikit-learn is employed to compute these similarity scores.

The function identifies the num_recommendations most similar songs to the input song based on their audio features. It does this by sorting the similarity scores in descending order and selecting the top num_recommendations songs. The input song itself is excluded from the recommendations (hence the [1:num_recommendations + 1] slicing). The function then extracts the details (such as track name, artists, album name, release date, and popularity) of the most similar songs from the music_df DataFrame using the indices of the most similar songs.

### 3.2 Function to get hybrid recommendations

In [ ]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [ ]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec

    # Check if hybrid_recommendations is a DataFrame before appending
    if isinstance(hybrid_recommendations, pd.DataFrame):
        hybrid_recommendations = hybrid_recommendations.append({
            'Track Name': input_song_name,
            'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
            'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
            'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
            'Popularity': weighted_popularity_score
        }, ignore_index=True)
    else:
        print("Error: content_based_rec is not a DataFrame.")

    # Sort the hybrid recommendations based on weighted popularity score
    if isinstance(hybrid_recommendations, pd.DataFrame):
        hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

        # Remove the input song from the recommendations
        hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

        return hybrid_recommendations
    else:
        return pd.DataFrame()  # Return an empty DataFrame in case of error


The hybrid approach aims to provide more personalized and relevant recommendations by considering both the content similarity of songs and their weighted popularity. The function takes input_song_name as the input, representing the name of the song for which recommendations are to be generated. The function first calls the content_based_recommendations function to get content-based recommendations for the input song. The num_recommendations parameter determines the number of content-based recommendations to be retrieved.

The function calculates the popularity score of the input song by retrieving the popularity value from the music_df DataFrame. It also calculates the weighted popularity score using the calculate_weighted_popularity function (previously defined) based on the release date of the input song. The alpha parameter controls the relative importance of content-based and popularity-based recommendations.

The content-based recommendations obtained earlier are stored in the content_based_rec DataFrame. The function combines the content-based recommendations with the input song’s information (track name, artists, album name, release date, and popularity) and its weighted popularity score. This step creates a DataFrame named hybrid_recommendations that includes both the content-based recommendations and the input song’s data.

The hybrid_recommendations DataFrame is then sorted in descending order based on the weighted popularity score. This step ensures that the most popular and relevant songs appear at the top of the recommendations. The input song is then removed from the recommendations to avoid suggesting the same song as part of the recommendations.

In [ ]:
input_song_name = "I'm Good (Blue)"

In [ ]:
# Llamada a la función content_based_recommendations
content_based_rec = content_based_recommendations(input_song_name, 5)

# Verificar si content_based_rec es un DataFrame
if isinstance(content_based_rec, pd.DataFrame):
    print("content_based_rec es un DataFrame")
else:
    print("content_based_rec NO es un DataFrame")


In [ ]:
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)